# AdminGPT: Your AI-Powered Administrative Assistant, powered by OpenAI's Assistant Framework  🚀
### Introduction
AdmiGPT is an AI-powered administrative assistant, harnessing the power of OpenAI's Assistant framework to seamlessly integrate with your email and calendar. Similar to Microsoft's Copilot, only better, it's designed to be your ultimate productivity partner, AdmiGPT offers an array of advanced features, making your administrative tasks simpler, faster, and more efficient.

AdminGPT is fully Open Source, so everything you need to run it for yourself is in this Github repo. This notebook helps you get started with AdminGPT, and walks you through how it's implemented. 

### Implementation
To begin, we're going to load our custom OpenAI Tools, which will interface with your email platform's API, and store any confidential and authentication information for the user in environmental variables.

In [1]:
from openai import OpenAI
import time, json, pprint, os
from tools.o365_toolkit import (
    o365search_emails,
    o365search_email,
    o365search_events,
    o365parse_proposed_times,
    o365send_message,
    o365reply_message,
    o365send_event,
    o365find_free_time_slots,
    tools,
)
from datetime import datetime as dt
from tools.utils import authenticate

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR OPENAI KEY"
# Set your Microsoft Graph client ID
os.environ["CLIENT_ID"] = "YOUR CLIENT ID"
# Set your Microsoft Graph client secret
os.environ["CLIENT_SECRET"] = "YOUR CLIENT SECRET"

Next, we're going to set a few constants, which we will use throughout the code.

In [2]:
account = authenticate()
mailbox = account.mailbox()
mailboxsettings = mailbox.get_settings()
timezone = mailboxsettings.timezone
directory = account.directory(resource="me")
user = directory.get_current_user()
client_name = user.full_name
client_email = user.mail

# Set values for global variables
assistant_name = "Monica A. Ingenio"
current_date = dt.now()
formatted_date = current_date.strftime("%A, %B %d, %Y")
openai_api_key = os.environ.get("OPENAI_API_KEY")
debug = False
model = "gpt-4-1106-preview"
assistant_instructions = (
    "You are an AI Administrative Assistant called "
    + assistant_name
    + ", and I am your executive. My name is "
    + client_name
    + ". My email is "
    + client_email
    + ", and I am in the "
    + timezone
    + " timezone. You have access to my email and calendar. Today is "
    + formatted_date
    + ". ALWAYS use functions for determining free times and parsing proposed"
    " meeting times in emails."
)

# Add the debug prompt if user runs with debug
if debug:
    debug_prompt = (
        "Keep a record of any feedback requests provided by me"
        " detailing the prompt and tools calls in case I want to retrieve"
        " them."
    )
else:
    debug_prompt = ""
assistant_instructions += debug_prompt

The next step is to create an OpenAI Assistant so we can interact with it, and a thread in which to run our prompts.

In [3]:
client = OpenAI(
    api_key=openai_api_key,
)

assistant = client.beta.assistants.create(
    name="AI Administrative Assistant",
    instructions=assistant_instructions,
    model=model,
    tools=tools,
)

thread = client.beta.threads.create()

To make execution easier in the future steps, we create a function to run prompts with only one call. Then, we are going to use the function to submit the initial coaching prompt so that the Assistant gets better at performing tasks from the beginning. (A big part of the secret sauce of AdminGPT is in the coaching data file called coaching_data.txt, which gives table-stakes knowledge to the model)

In [4]:
def run_prompt(prompt, client, assistant, thread):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    return run


with open("coaching_data.txt", "r") as file:
    prompt = file = file.read()

run = run_prompt(prompt, client, assistant, thread)

To further make execution easier in future steps, we Create a function that polls the OpenAI API for a response to the prompt and executes tool calls, so we can use it to retrieve responses in the future.

In [5]:
def poll_for_response(client, thread, run, model, debug=False):
    LOOP_DELAY_SECONDS = 3

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status

        if status == "completed":
            response = client.beta.threads.messages.list(thread_id=thread.id)
            if response.data:
                return response.data[0].content[0].text.value
            break
        elif status == "requires_action":
            tools_outputs = []

            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                function_arguments = tool_call.function.arguments
                function_arguments = json.loads(function_arguments)

                # Case statement to execute each toolkit function
                if function_name == "o365search_emails":
                    output = o365search_emails(**function_arguments)
                elif function_name == "o365search_email":
                    output = o365search_email(**function_arguments)
                elif function_name == "o365search_events":
                    output = o365search_events(**function_arguments)
                elif function_name == "o365parse_proposed_times":
                    output = o365parse_proposed_times(
                        **function_arguments, client=client, model=model
                    )
                elif function_name == "o365send_message":
                    output = o365send_message(**function_arguments)
                elif function_name == "o365send_event":
                    output = o365send_event(**function_arguments)
                elif function_name == "o365reply_message":
                    output = o365reply_message(**function_arguments)
                elif function_name == "o365find_free_time_slots":
                    output = o365find_free_time_slots(**function_arguments)

                # Clean the function output into JSON-like output
                output = pprint.pformat(output)
                tool_output = {"tool_call_id": tool_call_id, "output": output}
                tools_outputs.append(tool_output)

            if run.required_action.type == "submit_tool_outputs":
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tools_outputs
                )

        elif status == "failed":
            return "Run failed try again!"
            break

        if debug:
            print("The Assistant's Status is: " + status)

        time.sleep(LOOP_DELAY_SECONDS)

Now that we've submit our first prompt, we are going to poll for a response. If the response is, "How can I help you?", we know that the coaching data was sent correctly, and we are ready to start using AdminGPT.

In [6]:
response = poll_for_response(client, thread, run, model, debug)
print(response)

Hello, Santiago Delgado. How can I help you?


To begin, we're going to perform the most simple task, which is summarizing an email from a specific sender.

In [7]:
prompt = (
    "Can you please concisely summarize the most recent email from Santiago Delgado"
    " including any proposed meeting times?"
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

The most recent email from Santiago Delgado touches on the topic of Mike Portnoy potentially rejoining Dream Theater and asks for your thoughts on the matter. The sender suggests discussing this on February 3, 2024, at 4:00 pm ET. The email reflects on Portnoy's history with the band, his departure in 2010, and his influence on their classic sound. The sender is looking forward to hearing your thoughts on whether Portnoy's return could revitalize the classic Dream Theater sound or lead to complications.


Now, we're going to show AdminGPT's ability to interact with your calendar. We're going to check what events we have on the day that the email proposed.

In [8]:
prompt = (
    "What meetings do I have on the day that Santiago Delgado is proposing to meet in"
    " his most email?"
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

On February 3, 2024, the day Santiago Delgado proposed to meet in his email, you have the following meetings scheduled:

1. A flight from Dallas to Cincinnati from 8:00 am to 10:30 am ET.
2. A lunch meeting with management from 12:00 pm to 1:00 pm ET.
3. A strategy session from 3:00 pm to 5:00 pm ET.
4. A flight from Cincinnati to Dallas from 6:00 pm to 8:30 pm ET.

The proposed meeting time in Santiago Delgado's email, 4:00 pm ET, conflicts with the strategy session you have scheduled from 3:00 pm to 5:00 pm ET.


We now know when the email sender wants to meet, and what meetings I have on that day. So, let's see what times I have free that day.

In [9]:
prompt = "What times am I free on the day that Santiago is proposing to meet?"
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

On February 3, 2024, you have free time during the following periods:

1. From 12:00 am to 8:00 am ET.
2. From 10:30 am to 12:00 pm ET.
3. From 1:00 pm to 3:00 pm ET.
4. From 5:00 pm to 6:00 pm ET.
5. From 8:30 pm to 11:59 pm ET.


With all this information, we can now draft a response to the email letting the sender know that we can't meet at the proposed time, and propose other times to meet.

In [10]:
prompt = (
    "Please draft a response to Santiago letting him know that I can't meet on the"
    " proposed times, and propose other free times on the same day. Show me the draft."
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I have drafted a response to Santiago:

---

Dear Santiago,

Thank you for reaching out with the discussion about Mike Portnoy and Dream Theater. Unfortunately, I am not available at 4:00 pm ET on February 3 as I have a strategy session scheduled during that time.

However, I am available to discuss this at other times on the same day. Here are some potential time slots:
- 10:30 am to 12:00 pm ET
- 1:00 pm to 3:00 pm ET
- 5:00 pm to 6:00 pm ET
- 8:30 pm to 11:59 pm ET

Please let me know if any of these times work for you or if there is an alternate time you had in mind.

Best regards,

Santiago Delgado

---

The draft has been saved with the subject "Mike Portnoy coming back to Dream Theater?" Would you like me to send it?


Sometimes, we will want the email to come from our AI administrative instead of the ourselves, so we can do that too.

In [11]:
prompt = "Can you edit the email and have it come from you on behalf of me?"
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I have updated the draft response to Santiago Delgado to come from me on your behalf:

---

Hi Santiago,

I hope this message finds you well. Unfortunately, Santiago will not be available for a discussion at the proposed time of 4:00 pm ET on February 3, as he has a strategy session scheduled at that time.

However, Santiago is available at the following times on the same day, and he would be happy to discuss then:
- 10:30 am to 12:00 pm ET
- 1:00 pm to 3:00 pm ET
- 5:00 pm to 6:00 pm ET
- 8:30 pm to 11:59 pm ET

Please let us know if these alternative times work for you, or if another time would be better for Santiago's schedule.

Best regards,

Monica A. Ingenio
Executive Assistant to Santiago Delgado

---

The draft has been updated and saved with the subject "Mike Portnoy coming back to Dream Theater?" Would you like me to send this email now?


To conclude, I want to show that you can follow all these steps in one prompt.

In [12]:
prompt = (
    "Can you draft a response to Santiago Delgado's most recent email from you on"
    " behalf of me letting him know if I can meet at the times he proposes, and if not,"
    " propose other times on the same day? Show me the draft."
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I have drafted a response to the most recent email from Santiago Delgado on your behalf:

---

Dear Santiago,

I hope this message finds you well. I am writing on behalf of Santiago Delgado to inform you that he will not be available for the discussion at the proposed time of 4:00 pm ET on February 3rd, due to a prior commitment involving a strategy session.

Santiago is available to discuss the matter of Mike Portnoy and Dream Theater at these alternative times on the same day:
- 10:30 am to 12:00 pm ET
- 1:00 pm to 3:00 pm ET
- 5:00 pm to 6:00 pm ET
- 8:30 pm onward until the end of the day

Please advise if any of these times align with your schedule or if there is a better time for Santiago.

Warm regards,

Monica A. Ingenio
Executive Assistant to Santiago Delgado

---

The draft has been saved with the subject "Mike Portnoy coming back to Dream Theater?" Would you like me to send it?
